In [1]:
pip install pycryptodome

     |████████████████████████████████| 2.0 MB 5.9 MB/s 


In [2]:
from Crypto.Hash import SHA256

In [3]:
def eea (a, b):
    if a == 0: return (b, 0, 1)
    else:
        (gcd, x1, y1) = eea(b%a, a)
        x = y1 - (b//a)*x1
        y = x1
        return (gcd, x, y)

def modinverse(a, b):
    return eea(a,b)[1] % b

In [4]:
from Crypto.PublicKey import RSA
rsa_key = RSA.generate(2048, e = 2**16+1)

In [5]:
p = rsa_key.p
q = rsa_key.q
e = rsa_key.e
n = rsa_key.n
d = rsa_key.d

Encryption and decryption keys for encrypting and decrypting our texts.

In [6]:
print(p)
print(q)
print(e)
print(n)
print(d)

137913865115385204939119513368160631239568714898345149849253654335467891017578318618637657430855716383347446015482893466856482796128800599538942544531808652914929975036231256369019246702610765781702792226896286198013748961609253598330646066137323714199225015525536579693095223510824441769936497160267475356503
160066044438621968691186376026270504357154393849610365615196297181257355943008578984654255948641742030341586913257526633526908344183923186095528787823892093728728557043277544116414599573982157787526728595888166799467734638904179668569842528412970569728770263560133629930155823626314154362190576503352589218143
65537
22075326862261364317455111649405497380118552536822666102965625545668629458554190297842492921161553479192494106682465908454759670528751792753553127217412982753652484266573938292802484479805435447380229761629125729454016201618025737318746626951747423877237845415974290615944188942781120671685174277983560440971347812099857118559101181171430039050954955069110339351365176980639

In [7]:
plain_text1 = 1234
blow_up_factor = 10000000

byte_stream1 = str(plain_text1).encode()

hash_text1 = SHA256.new(byte_stream1)

cipher_text1 = pow(plain_text1, e, n)
cipher_text2 = pow(blow_up_factor, e, n)

cipher_text_blow_up = (cipher_text1 * cipher_text2) % n

plain_text_blow_up = pow(cipher_text_blow_up, d, n)
print(plain_text_blow_up)
print(hash_text1)

byte_stream_blow_up = str(plain_text_blow_up).encode()

hash_blow_up = SHA256.new(byte_stream_blow_up)

if hash_blow_up != hash_text1:
  print("alert! transaction amt altered")
else:
  print("success relax")

12340000000
alert! transaction amt altered


Use authentication to prevent a malleability attack. Authentication works by sending a hashed version(hash_text1) of the original amt(plain_text1) and the encrypted original amt(cipher_text1).If the hashed amt(hash_text1) equals the hashed encrypted amt (hash_blow_up) then there is no attack. In this case there is a malleability attack. The original amt(plain_text1) has been blown up using the blow_up_factor. To prevent the attack we decrypt the sent encrypted amt (cipher_text_blow_up) into plain_text_blow_up. We then hash plain_text_blow_up. Since we have the original hashed number(hash_text1), we then see if hash_blow_up is equal to hash_text1. We see that they aren't and we send an alert. We also see that multiplication is homomorphic while encrypting and decrypting.

In [8]:
plain_text1 = 12345678
plain_text2 = 10000000

cipher_text1 = pow(plain_text1, e, n)
cipher_text2 = pow(plain_text2, e, n)

cipher_text_sum = (cipher_text1 + cipher_text2) % n

plain_text_sum = pow(cipher_text_sum, d, n)
print(plain_text_sum)

14440101893160040831549797345610024779885821612341830906956911504115077014976893085222858254432620663118609090984604464938779537546573689094078993574521930781657698141186594508020444306614643378520819052510372491518942174290130540391189971236460586504329874725421653513961148401495035895083951296544660086218274862261662521040791402654506650384447863934284086058068026247321277807743040281602365246970695479079451604744993385301455180068663249544647498905760492232449144478601891977957439703466026538627961466717542305846857360226790056332031035559930084772431933225119343423597274072477370348827627755546963010277369


Addition is not homomorphic.

In [9]:
plain_text1 = 12345678
plain_text2 = 10000000

cipher_text1 = pow(plain_text1, e, n)
cipher_text2 = pow(plain_text2, e, n)

cipher_text_sum = (cipher_text1 - cipher_text2) % n

plain_text_sum = pow(cipher_text_sum, d, n)
print(plain_text_sum)

10929277064473432248550446507960778764820894316106009984733994408389339674321274998358650516120828255762249179639296497396974735898624706531361696731600003038450203394595132605986896598331464096870988385190042941685673642813565749719742280522530449455370023737910488751363065765879608732774836600409041773663915549700392234177976830379679950345650962749578246230437751454354569103083432300019369128109102854294680292714880776093623427510631167516955413665187627904581205016226322660810067873427295711188818063643148954180138171749725322209361240664242512628598071857684962272253012131463914402945099548497594259824089


Subtraction is not homomorphic.